# Criação de views para facilitar a análise
  **Views** e **tabelas agregadas** são essenciais para otimizar o acesso aos dados, melhorar o desempenho das consultas, centralizar a lógica de negócios e facilitar a análise e criação de métricas em sistemas de BI. Elas proporcionam uma maneira mais eficiente e segura de trabalhar com grandes volumes de dados e são ferramentas indispensáveis na entrega de *insights* precisos e rápidos para as equipes de negócios.

## vw_fato_vendas

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_fato_vendas` AS
SELECT
    f.id_venda,
    c.cliente,
    fi.filial,
    fi.municipio,
    fi.uf,
    pr.item,
    pr.categoria,
    pr.subcategoria,
    EXTRACT(YEAR FROM f.data_venda) AS ano,
    EXTRACT(MONTH FROM f.data_venda) AS mes,
    EXTRACT(DAY FROM f.data_venda) AS dia,
    DATE(f.data_venda) AS dt_venda,
    f.vr_unitario,
    f.vr_total_pago
FROM `datawarehouse.fato_venda` f
INNER JOIN `datawarehouse.dim_item` pr ON f.id_item = pr.id_item
INNER JOIN `datawarehouse.dim_associado` pa ON f.id_associado = pa.id_associado
INNER JOIN `datawarehouse.dim_filial` fi ON f.id_filial = fi.id_filial
INNER JOIN `datawarehouse.dim_cliente` c ON f.id_cliente = c.id_cliente

In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vw_fato_vendas`
LIMIT 5

## vw_fato_vendas_tempo_cliente_item

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_fato_vendas_tempo_cliente_item` AS
SELECT
    data_venda,
    EXTRACT(YEAR FROM data_venda) AS ano,
    EXTRACT(MONTH FROM data_venda) AS mes,
    id_cliente,
    id_item,
    SUM(vr_unitario * qtde) AS total_vendas,
    COUNT(DISTINCT id_venda) AS qtd_vendas,
    AVG(vr_unitario * qtde) AS ticket_medio
FROM `datawarehouse.fato_venda`
GROUP BY data_venda, id_cliente, id_item;


In [ ]:
%%bigquery
SELECT *
FROM `vw_fato_vendas_tempo_cliente_produto`
LIMIT 10

In [ ]:
%%bigquery
SELECT *
FROM `vw_vendas_tempo_produto`
LIMIT 10

## vw_vendas_localizacao_cliente

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_vendas_localizacao_cliente` AS
SELECT
    l.uf,
    l.municipio,
    l.id_filial,
    l.filial,
    c.id_cliente,
    c.cliente,
    c.premium,
    SUM(f.qtde) AS total_qtde,
    SUM(f.vr_total_pago) AS total_vendas,
    AVG(f.vr_unitario) AS media_vr_unitario
FROM `datawarehouse.fato_venda` f
JOIN `datawarehouse.dim_filial` l ON f.id_filial = l.id_filial
JOIN `datawarehouse.dim_cliente` c ON f.id_cliente = c.id_cliente
GROUP BY l.uf, l.municipio, l.id_filial, l.filial,
c.id_cliente, c.cliente, c.premium

## vw_produto_subcategoria_categoria


In [ ]:
%%bigquery
CREATE VIEW `datawarehouse.vw_hierarquia_item_categoria` AS
SELECT
  categoria,
  subcategoria,
  item,
  ROUND(SUM(vr_total_pago),2) AS total_vendas
FROM `datawarehouse.fato_venda` f
INNER JOIN `datawarehouse.dim_item` p ON f.id_item = p.id_item
GROUP BY categoria, subcategoria, item

# Tabelas pré agregadas com métricas comuns

## Criação da tabela: vendas_categoria_ano

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.vendas_categoria_ano` AS
SELECT
  p.categoria,
  --EXTRACT(YEAR FROM PARSE_DATE('%Y%m%d', CAST(f.data_venda AS STRING))) AS ano,
  EXTRACT(YEAR FROM f.data_venda) AS ano,
  SUM(f.vr_total_pago) AS total_vendas
FROM `datawarehouse.fato_venda` f
JOIN `datawarehouse.dim_item` p ON f.id_item = p.id_item
GROUP BY p.categoria, ano;

In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vendas_categoria_ano`
LIMIT 5

## Criação da tabela: vendas_diarias

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.vendas_diarias` AS
WITH tmp_total_vendas AS (
  SELECT
    f.data_venda,
    ROUND(SUM(f.vr_unitario * f.qtde),2) AS total_vendas,
    COUNT(f.id_venda) as qtdade_vendas,
    --AVG(SUM(f.vr_unitario * f.qtde)) OVER (GROUP BY f.id_venda) AS media
  FROM `datawarehouse.fato_venda` f
  GROUP BY  f.data_venda
)
SELECT
  data_venda,
  total_vendas,
  qtdade_vendas,
  round(total_vendas/qtdade_vendas, 2) as media
FROM tmp_total_vendas


In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vendas_diarias`
LIMIT 5

# Cálculo das comissões por parceiro

## Criação da procedure que calcula a comissão bruta e liquida por parceiro mensalmente

In [ ]:
%%bigquery
CREATE OR REPLACE PROCEDURE `datawarehouse.p_fato_comissao_mensal`()
BEGIN

  CREATE OR REPLACE TABLE `datawarehouse.comissao_mensal` AS
  SELECT
    f.id_associado,
    pa.associado as associado,
    FORMAT_DATE('%Y-%m', f.data_venda) AS mes_ano,
    ROUND(SUM(f.vr_total_pago),2) AS total_vendas,
    ROUND(SUM(f.qtde * ((f.vr_unitario * p.percentual)/100)),2) AS comissao_bruta,
    0.0 as bonus_aplicado, -- força o dado a ser do tipo float
    0.0 as comissao_final
  FROM `datawarehouse.fato_venda` f
  INNER JOIN `datawarehouse.dim_item` p ON f.id_item = p.id_item
  INNER JOIN `datawarehouse.dim_associado` pa ON f.id_associado = pa.id_associado
  GROUP BY f.id_associado, pa.associado, mes_ano;

  -- Atualiza o valor da comissão bruta e comissão final
  UPDATE `datawarehouse.comissao_mensal`
  SET
    bonus_aplicado =
      CASE
        WHEN (total_vendas >= 10000) AND (comissao_bruta >= 100) THEN 100 -- caso a comissão seja maior que 100
        WHEN (total_vendas >= 10000) AND (comissao_bruta < 100) THEN comissao_bruta -- caso a comissão seja menor do que 100, o desconto é total
        ELSE 0
      END,

    comissao_final =
      CASE
        WHEN (total_vendas >= 10000) AND (comissao_bruta >= 100) THEN ROUND(comissao_bruta - 100, 2) -- valor final da comissão subtraído os 100 de bonus
        WHEN (total_vendas >= 10000) AND (comissao_bruta < 100) THEN 0 -- zerando a comissao
        ELSE comissao_bruta
      END
  WHERE TRUE;

  SELECT
    associado,
    mes_ano,
    total_vendas,
    comissao_bruta,
    bonus_aplicado,
    comissao_final
  FROM `datawarehouse.comissao_mensal`
  ORDER BY 1, 2;
END;


## Execução da procedure que calcula as comissão bruta e liquida por parceiro mensalmente

In [ ]:
%%bigquery
CALL `datawarehouse.p_fato_comissao_mensal`()

## Cálculo da comissão mensal por associado


In [ ]:
%%bigquery
CREATE OR REPLACE TABLE `datawarehouse.comissao_mensal_associado` AS
SELECT
  id_associado,
  associado,
  mes_ano,
  total_vendas,
  comissao_bruta,
  bonus_aplicado,
  comissao_final,
  ROUND(((comissao_bruta / total_vendas) * 100), 2) AS percentual_comissao_bruta,
  ROUND(((comissao_final / total_vendas) * 100), 2) AS percentual_comissao_liquida,
FROM `datawarehouse.comissao_mensal`
ORDER BY 1, 2;

### Comparação da venda de um dia D com a média móvel dos últimos 30 dias anteriores à D.

A Análise de Tendência de Vendas com Média Móvel é uma das ferramentas mais valiosas para identificar padrões e avaliar o desempenho de vendas ao longo do tempo. Ao comparar as vendas de um dia específico (Dia D) com a média móvel dos últimos 30 dias, você obtém uma visão mais clara e precisa da evolução das vendas, levando em consideração flutuações naturais e atenuando picos ou quedas sazonais.

In [ ]:
%%bigquery
SELECT
  FORMAT_TIMESTAMP('%Y-%m-%d', data_venda) AS data_venda,
  media,
  ROUND(AVG(media) OVER (ORDER BY data_venda ROWS BETWEEN 30 PRECEDING AND CURRENT ROW), 2) AS media_movel_30d,
  ROUND(
    (
      (media - AVG(media) OVER (ORDER BY data_venda ROWS BETWEEN 30 PRECEDING AND CURRENT ROW)) /
      AVG(media) OVER (ORDER BY data_venda ROWS BETWEEN 30 PRECEDING AND CURRENT ROW ) * 100
    ), 2
  ) AS variacao_percentual
FROM `datawarehouse.vendas_diarias`

### Comparação da venda do dia atual com a venda do dia anterior

Essa informação possibilita monitorar a performance diária de vendas e identificar flutuações ou tendências imediatas.

In [ ]:
%%bigquery
SELECT
  FORMAT_TIMESTAMP('%Y-%m-%d', t.data_venda) AS data_venda,
  t.total_vendas,
  t.media,
  FORMAT_TIMESTAMP('%Y-%m-%d', TIMESTAMP_SUB(t.data_venda, INTERVAL 1 DAY)) AS dia_anterior,
  IFNULL(tvd.media, 0) AS media_dia_anterior,
  CASE
    WHEN tvd.media = 0 THEN 0 -- caso não exista registros do dia anterior o valor será 0
    ELSE ROUND(((t.media / tvd.media) * 100), 2)
  END AS variacao_percentual
FROM `datawarehouse.vendas_diarias` t
LEFT JOIN `datawarehouse.vendas_diarias` tvd
  ON tvd.data_venda = TIMESTAMP_SUB(t.data_venda, INTERVAL 1 DAY)
ORDER BY t.data_venda
LIMIT 5

### Soma acumulada das vendas de um parceiro no mês

Essa métrica é importante porque oferece uma visão clara e consolidada do desempenho de vendas de um parceiro ao longo de um período específico, permitindo uma série de análises e decisões estratégicas.

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_vendas_associado_mes` AS
SELECT
  pa.associado,
  FORMAT_DATE('%Y-%m', f.data_venda) AS mes_ano,
  ROUND(SUM(f.vr_unitario * f.qtde),2) AS total_vendas,
FROM `datawarehouse.fato_venda` f
INNER JOIN `datawarehouse.dim_associado` pa ON pa.id_associado = f.id_associado
GROUP BY pa.associado, mes_ano
ORDER BY pa.associado, mes_ano;

SELECT *
FROM `datawarehouse.vw_vendas_associado_mes`

### Soma acumulada das vendas de um parceiro por categoria no mês

Fornece insights detalhados sobre o desempenho de vendas em diferentes segmentos de produto ou serviço ao longo do mês.

In [ ]:
%%bigquery
--CREATE OR REPLACE VIEW `datawarehouse.vw_soma_associado_categoria_mensal` AS

SELECT
  pa.associado,
  FORMAT_DATE('%Y-%m', f.data_venda) AS mes_ano,
  p.categoria,
  ROUND(SUM(f.vr_unitario * f.qtde),2) AS total_vendas,
FROM `datawarehouse.fato_venda` f
INNER JOIN `datawarehouse.dim_item` p ON f.id_item = p.id_item
INNER JOIN `datawarehouse.dim_associado` pa ON pa.id_associado = f.id_associado
GROUP BY  pa.associado,
          mes_ano,
          p.categoria
ORDER BY pa.associado, mes_ano, p.categoria

### Calcule o imposto mensal aplicando uma alíquota de 7,5% sobre o valor das vendas, e rateie proporcionalmente entre os parceiros com base no valor da comissão de cada um. Certifique-se de que os impostos de um mês não sejam distribuídos em outro.


In [ ]:
## Para fim de verificação calculei o imposto_mensal_geral
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_imposto_mensal_geral` AS
SELECT
  FORMAT_DATE('%Y-%m', f.data_venda) AS mes,
  ROUND(SUM(f.vr_unitario * f.qtde),2) AS total_vendas,
  ROUND(((SUM(f.vr_unitario * f.qtde) * 7.5)/100), 2) AS imposto
FROM `datawarehouse.fato_venda` f
INNER JOIN `datawarehouse.dim_associado` pa ON pa.id_associado = f.id_associado
GROUP BY mes
ORDER BY mes

In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vw_imposto_mensal_geral`


In [ ]:
%%bigquery
CREATE OR REPLACE VIEW `datawarehouse.vw_imposto_associado_rateado` AS
  SELECT
    associado as associado,
    mes_ano,
    total_vendas,
    comissao_final,
    ROUND((total_vendas * 9.25) / 100, 2) AS imposto_total,
    ROUND(
      (comissao_final / SUM(comissao_final) OVER (PARTITION BY mes_ano)) * ((total_vendas * 9.25) / 100),
      2
    ) AS imposto_rateado
  FROM `datawarehouse.comissao_mensal_associado`
  ORDER BY associado, mes_ano;

In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vw_imposto_parceiro_rateado`

In [ ]:
%%bigquery
SELECT *
FROM `datawarehouse.vw_imposto_parceiro`